In [0]:
import numpy as np
from keras.callbacks import ModelCheckpoint
from keras.layers import Conv2D, Flatten, MaxPooling2D, Dense, Dropout
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img
import random, os, glob
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [8]:
! unzip data

Archive:  data.zip
   creating: data/cardboard/
  inflating: data/cardboard/cardboard1.jpg  
  inflating: data/cardboard/cardboard10.jpg  
  inflating: data/cardboard/cardboard100.jpg  
  inflating: data/cardboard/cardboard101.jpg  
  inflating: data/cardboard/cardboard102.jpg  
  inflating: data/cardboard/cardboard103.jpg  
  inflating: data/cardboard/cardboard104.jpg  
  inflating: data/cardboard/cardboard105.jpg  
  inflating: data/cardboard/cardboard106.jpg  
  inflating: data/cardboard/cardboard107.jpg  
  inflating: data/cardboard/cardboard108.jpg  
  inflating: data/cardboard/cardboard109.jpg  
  inflating: data/cardboard/cardboard11.jpg  
  inflating: data/cardboard/cardboard110.jpg  
  inflating: data/cardboard/cardboard111.jpg  
  inflating: data/cardboard/cardboard112.jpg  
  inflating: data/cardboard/cardboard113.jpg  
  inflating: data/cardboard/cardboard114.jpg  
  inflating: data/cardboard/cardboard115.jpg  
  inflating: data/cardboard/cardboard116.jpg  
  inflating: dat

In [0]:
dir_path = "data/"

In [0]:
# - The glob module finds all the pathnames matching a specified pattern according to the rules used by the Unix shell
# - os implements some useful functions on pathnames. To read or write files see open(), and for accessing the filesystem see the os module. 
# The path parameters can be passed as either strings, or bytes. Applications are encouraged to represent file names as (Unicode) character strings
img_list = glob.glob(os.path.join(dir_path + '*/*.jpg'))

In [11]:
# Verificando tamanho da lista para bater com o total de fotos no dataset - 2527
print(len(img_list))
# Verificando tipos de variaveis 
print(type(img_list))
print(type(img_list[0]))
print(img_list[0])
# Portanto, nesse primeiro Array, capturamos somente o caminho até as imagens

2527
<class 'list'>
<class 'str'>
data/plastic/plastic393.jpg


In [12]:
# Generate batches of tensor image data with real-time data augmentation. The data will be looped over (in batches).
train = ImageDataGenerator(horizontal_flip = True, 
                           vertical_flip = True, 
                           validation_split = 0.1, 
                           rescale = 1./255, 
                           shear_range = 0.1, 
                           zoom_range = 0.1, 
                           width_shift_range = 0.1,
                           height_shift_range = 0.1
                          )
test = ImageDataGenerator(rescale = 1/255, validation_split = 0.1)
# Lovely flow_from_directory article! https://medium.com/@vijayabhaskar96/tutorial-image-classification-with-keras-flow-from-directory-and-generators-95f75ebe5720

train_generator = train.flow_from_directory(dir_path, target_size = (300, 300), batch_size = 32, class_mode = 'categorical', subset='training')

test_generator = test.flow_from_directory(dir_path, target_size = (300, 300), batch_size = 32, class_mode = 'categorical', subset='validation')

Found 2276 images belonging to 6 classes.
Found 251 images belonging to 6 classes.


In [0]:
# Makes the classes Dictonary easier to handle
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())

In [0]:
# artigo para entender melhor o que essas layer fazem
# https://machinelearningmastery.com/how-to-configure-the-number-of-layers-and-nodes-in-a-neural-network/
# Artigo sobre redes neurais aplicadas a imagens - Muito importante para entender nosso problema.
# https://towardsdatascience.com/a-comprehensive-guide-to-convolutional-neural-networks-the-eli5-way-3bd2b1164a53

In [15]:
model = Sequential()
# Input Layer
model.add(Conv2D(32, (3, 3), padding = 'same', input_shape = (300, 300, 3), activation = 'relu'))
# Hidden Layers
#   Convolutional
model.add(MaxPooling2D(pool_size = 2))
model.add(Conv2D(64, (3, 3), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = 2))
model.add(Conv2D(32, (3, 3), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = 2))
#   Classification
model.add(Flatten())
model.add(Dense(64, activation = 'relu'))
# Output Layer
model.add(Dense(6, activation = 'softmax'))

filepath = 'trained_model.h5'
checkpoint1 = ModelCheckpoint(filepath, monitor = 'val_acc', verbose = 1, save_best_only = True, mode = 'max')
callbacks_list = [checkpoint1]

In [16]:
# Sumário das diferentes camadas da rede neural e seus autputs
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 300, 300, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 150, 150, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 150, 150, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 75, 75, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 75, 75, 32)        18464     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 37, 37, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 43808)            

In [17]:
model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['acc'])

In [0]:
model.fit_generator(train_generator, 
                    epochs = 100, 
                    steps_per_epoch = 2275//32, 
                    validation_data = test_generator,
                    validation_steps = 251//32,
                    callbacks = callbacks_list)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/100
71/71 [==============================] - 350s 5s/step - loss: 1.5336 - acc: 0.3240 - val_loss: 1.5057 - val_acc: 0.3438

Epoch 00001: val_acc improved from -inf to 0.34375, saving model to trained_model.h5
Epoch 2/100
71/71 [==============================] - 348s 5s/step - loss: 1.3018 - acc: 0.4652 - val_loss: 1.2852 - val_acc: 0.4932

Epoch 00002: val_acc improved from 0.34375 to 0.49315, saving model to trained_model.h5
Epoch 3/100
71/71 [==============================] - 348s 5s/step - loss: 1.2180 - acc: 0.5335 - val_loss: 1.2483 - val_acc: 0.4886

Epoch 00003: val_acc did not improve from 0.49315
Epoch 4/100
71/71 [==============================] - 346s 5s/step - loss: 1.1114 - acc: 0.5620 - val_loss: 1.4550 - val_acc: 0.3927

Epoch 00004: val_acc did not improve from 0.49315
Epoch 5/100
71/71 [==============================] - 346s 5s/step - loss: 1.1171 - acc: 0.5704 - val